In [2]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pymysql
import mysql
import mysql.connector
import pandas as pd

In [16]:
import mysql.connector

try:
    db_connection = pymysql.connect(
        host="localhost",
        user="coursera",
        password="1234567890"
    )
    print("Connected to MySQL successfully.")
except Exception as e:
    print(f"Error: {e}")

Error: (2003, "Can't connect to MySQL server on 'localhost' ([WinError 10061] No connection could be made because the target machine actively refused it)")


In [51]:

# Create the Database
cursor.execute("CREATE DATABASE IF NOT EXISTS course_data")
cursor.execute("USE course_data")

# Create Tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS Organizations (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255) UNIQUE NOT NULL)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Certificate_Types (
    id INT AUTO_INCREMENT PRIMARY KEY,
    type VARCHAR(255) UNIQUE NOT NULL)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Courses (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    organization_id INT NOT NULL,
    certificate_type_id INT NOT NULL,
    rating FLOAT,
    reviews INT,
    coursera_plus BOOLEAN,
    difficulty VARCHAR(50),
    duration VARCHAR(50),
    FOREIGN KEY (organization_id) REFERENCES Organizations(id),
    FOREIGN KEY (certificate_type_id) REFERENCES Certificate_Types(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Skills (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255) UNIQUE NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Course_Skills (
    course_id INT NOT NULL,
    skill_id INT NOT NULL,
    PRIMARY KEY (course_id, skill_id),
    FOREIGN KEY (course_id) REFERENCES Courses(id),
    FOREIGN KEY (skill_id) REFERENCES Skills(id)
)
""")

# Insert Organizations
organizations = courses_df_clean["Organization"].unique()
for org in organizations:
    cursor.execute("""
        INSERT IGNORE INTO Organizations (name) VALUES (%s)
    """, (org,))

# Insert Certificate Types
certificate_types = courses_df_clean["Certificate Type"].unique()
for cert_type in certificate_types:
    cursor.execute("""
        INSERT IGNORE INTO Certificate_Types (type) VALUES (%s)
    """, (cert_type,))

# Insert Courses
for _, row in courses_df_clean.iterrows():
    cursor.execute("""
        INSERT INTO Courses (title, organization_id, certificate_type_id, rating, reviews, coursera_plus, difficulty, duration)
        VALUES (
            %s,
            (SELECT id FROM Organizations WHERE name = %s),
            (SELECT id FROM Certificate_Types WHERE type = %s),
            %s, %s, %s, %s, %s
        )
    """, (row["Title"], row["Organization"], row["Certificate Type"], row["Rating"], row["Reviews"],
          row["Coursera Plus"], row["difficulty"], row["Duration"]))

# Insert Skills
all_skills = set(skill for skill_list in courses_df_clean["Skills"] for skill in skill_list)
for skill in all_skills:
    cursor.execute("""
        INSERT IGNORE INTO Skills (name) VALUES (%s)
    """, (skill,))

# Insert Course-Skill Mapping
for _, row in courses_df_clean.iterrows():
    for skill in row["Skills"]:
        cursor.execute("""
            INSERT INTO Course_Skills (course_id, skill_id)
            VALUES (
                (SELECT id FROM Courses WHERE title = %s),
                (SELECT id FROM Skills WHERE name = %s)
            )
        """, (row["Title"], skill))

# Commit changes and close connection
connection.commit()
cursor.close()
connection.close()

print("Data loaded successfully.")


NameError: name 'cursor' is not defined